In [1]:
# TensorFlow 라이브러리를 사용한 로지스틱(logistic) 회귀 학습 알고리즘 예제.
# 이 예제는 손으로 쓴 숫자들을 모와 둔 MNIST 데이터베이스를 사용하고 있습니다.
# (http://yann.lecun.com/exdb/mnist/)

# 작성자: Aymeric Damien
# 수정자: 정 경태
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
import tensorflow as tf

# MINST 데이터 가져오기
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# 파라메터(Parameters)
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf 그래프(Graph) 입력
x = tf.placeholder(tf.float32, [None, 784]) # mnist 데이터 이미지의 모양 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 숫자 인식 => 10개의 클래스

# 모델 가중치(weights) 설정
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# 모델 생성
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# cross entropy를 사용하여 오차 최소화
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# 변수 초기화
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [4]:
# 세션 설정
session_conf = tf.ConfigProto()
session_conf.gpu_options.allow_growth = True

# 그래프 실행
with tf.Session(config=session_conf) as sess:
    sess.run(init)

    # 훈련 주기
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # 모든 배치 반복
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # 배치 데이터를 사용하여 피팅(fit) 훈련
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 평균 손실 계산
            avg_cost += c / total_batch
        # 매 epoch 스텝 마다 화면에 로그 출력
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("최적화 종료됨!")

    # 테스트 모델
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 3000 가지 예제에 대한 정확도 계산
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))

InternalError: Failed to create session.